# Incremental margin algorithm for large margin classifiers

In [2]:
# Imports
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_curve, auc
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import LinearSVC
from sklearn import preprocessing
from sklearn.datasets import load_digits
import time
from sklearn import svm
from sklearn.model_selection import GridSearchCV 
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold
from scipy.stats import sem
from numpy import linalg as LA
from copy import deepcopy
import warnings
warnings.filterwarnings("ignore")

## Calculating the margin

In [3]:
def compute_margin(X, y, w, b):
    margin = []
    for i in range(y.shape[0]):
        margin.append((y[i]*(np.dot(X[i,:], w)+b))/sqrt(sum(w**2))) 
    if min(margin) >= 0:
        return min(margin)
    return 0

## L0 norm

In [4]:
def L0_norm(w, threshold):
    l0_norm = 0
    for wi in w:
        if abs(wi) > threshold:
            l0_norm += 1
    return l0_norm

## ELM IMA p=2

In [5]:
import random
from sklearn.base import BaseEstimator, ClassifierMixin
class ELM_IMA(BaseEstimator, ClassifierMixin):

    # Inicialization of important parameters 
    def __init__(self, n_neurons, eta=0.1, lambda_param=0.01, delta_margin=10^-3,
                 IMA_iterations=10, max_updates=10000):
        self.n_neurons = n_neurons              # Neurons of hidden layer osf ELM
        self.eta = eta                          # Learning rate
        self.lambda_param = lambda_param        # Param important of soft margin
        self.delta_margin = delta_margin        # (1 + delta_margin) * fixed margin defines the minimum next margin of IMA
        self.IMA_iterations = IMA_iterations    # Maximum number of iterations of IMA
        self.max_updates = max_updates          # Maximum number of updates in one execution of FMP
        self.w = np.array([])                   # Vector of weights of the last layer of the ELM obtained after the training of the IMA
        self.w_elm = np.array([])               # Vector of weights of the last layer of the ELM obtained after the normal training of ELM
        self.H = np.array([])                   # H matrix of ELM (obtained with training data)
        self.Z = np.array([])                   # Z matrix of ELM
        self.b = 0

    # Fixed Margin Algorithm    
    def FMP_algorithm(self, X, y, w_init, b_init, fixed_margin, idx, s):
        t = 0
        iterations = 0
        w = w_init
        b = b_init
        norm_w = sqrt(sum(w**2))
        last_t = -1
        lambda_t = 0
        alpha = np.zeros((X.shape[0]))
        while True:
            last_t = t
            e=0
            for k in range(0, y.shape[0]):
                i = int(idx[k])
                if(y[i]*(np.dot(X[i,:], w)+b) <= fixed_margin * norm_w - self.lambda_param * alpha[i]):
                    if norm_w != 0:
                        lambda_t = 1 - (self.eta*fixed_margin)/norm_w
                    else:
                        lambda_t = 1
                    alpha = alpha * lambda_t
                    alpha[i] = alpha[i] + self.eta    
                    w = w * lambda_t + self.eta * y[i] * X[i,:]
                    norm_w = sqrt(sum(w**2))
                    b = b + self.eta*y[i]
                    t += 1
                    e += 1
                    if k > s:
                        s += 1
                        j = s
                    else:
                        j=e
                    idx[k], idx[j] = idx[j], idx[k]
            iterations += 1
            if (t > self.max_updates or last_t == t):
                break
        if t<= self.max_updates:
            convergence=1
        else:
            convergence=0
        return w, b, convergence, t, iterations, idx, s

    # IMA Algorithm
    def IM_algorithm(self, X, y):
        self.w = np.zeros(self.H.shape[1])
        self.ws = [] 
        self.bs = [] 
        self.ws.append(self.w)
        self.bs.append(self.b)
        fixed_margin = 0#compute_margin(X, y, self.w_elm, self.b)
        t = 0
        convergence = 1
        updates=0
        iterations=0
        margin=[]
        margin.append(fixed_margin)
        idx = np.linspace(0, y.shape[0]-1, y.shape[0])
        s=0
        while convergence==1 and t<self.IMA_iterations:
            w, b, convergence, updates_, iterations_, idx, s = self.FMP_algorithm(X, y, self.w, self.b, fixed_margin, idx, s)
            if convergence == 1:
                self.w = w
                self.b = b
                self.ws.append(self.w)
                self.bs.append(self.b)
            updates += updates_
            iterations += iterations_
            norm_w = sqrt(sum(self.w**2))
            gamma1 = []
            gamma2 = []
            for i in range(0, y.shape[0]):
                if y[i] == 1:
                    gamma1.append((y[i]*(np.dot(X[i], self.w)+self.b))/norm_w)
                else:
                    gamma2.append((y[i]*(np.dot(X[i], self.w)+self.b))/norm_w)
            gamma1 = np.array(gamma1)
            gamma2 = np.array(gamma2)
            gamma1 = gamma1[gamma1>=0]
            gamma2 = gamma2[gamma2>=0]
            if len(gamma1) == 0:
                min_gamma1 = 0
            else:
                min_gamma1 = min(gamma1)
            if len(gamma2) == 0:
                min_gamma2 = 0
            else:
                min_gamma2 = min(gamma2)
            fixed_margin = max([(min_gamma1 + min_gamma2)/2, (1+self.delta_margin)*fixed_margin])
            #margin.append(compute_margin(X, y, self.w, self.b))
            t += 1
        return t, updates, iterations

    # Function that manage the training of IMA ELM
    def fit(self, X, y, Z=[]):
        X_new = np.ones((X.shape[0], X.shape[1]+1))
        X_new[:,1:] = X
        X = X_new
        n = X.shape[1]
        if len(Z) == 0:
            self.Z = np.array([random.uniform(-0.5, 0.5) for i in range(n*self.n_neurons)]).reshape(n, self.n_neurons)
        else:
            self.Z = Z
        self.H = np.tanh(np.dot(X, self.Z))
        #w = np.dot(np.linalg.pinv(self.H), y)  
        #self.w_elm = w.reshape((w.shape[0],))
        iterations_IMA, updates, iterations = self.IM_algorithm(self.H, y) 
        return iterations_IMA, updates, iterations
            
    # Function to apply IMA ELM model
    def predict(self, X, use_IMA_w=True):
        X_new = np.ones((X.shape[0], X.shape[1]+1))
        X_new[:,1:] = X
        H = np.tanh(np.dot(X_new, self.Z))
        if use_IMA_w == True:
            y_predicted = np.sign(np.dot(H, self.w) + self.b)
        else:
            y_predicted = np.sign(np.dot(H,  self.w_elm))
        y_predicted[y_predicted==0]=-1
        return y_predicted

## ELM with IMA p=inf

In [6]:
import random
from sklearn.base import BaseEstimator, ClassifierMixin

class ELM_IMA_inf(BaseEstimator, ClassifierMixin):

    # Inicialization of important parameters 
    def __init__(self, n_neurons, eta=0.1, lambda_param=0.01, delta_margin=10^-3,
                 IMA_iterations=10, max_updates=10000, threshold=0.1):
        self.n_neurons = n_neurons              # Neurons of hidden layer osf ELM
        self.eta = eta                          # Learning rate
        self.lambda_param = lambda_param        # Param important of soft margin
        self.delta_margin = delta_margin        # (1 + delta_margin) * fixed margin defines the minimum next margin of IMA
        self.IMA_iterations = IMA_iterations    # Maximum number of iterations of IMA
        self.max_updates = max_updates          # Maximum number of updates in one execution of FMP
        self.w = np.array([])                   # Vector of weights of the last layer of the ELM obtained after the training of the IMA
        self.w_elm = np.array([])               # Vector of weights of the last layer of the ELM obtained after the normal training of ELM
        self.H = np.array([])                   # H matrix of ELM (obtained with training data)
        self.Z = np.array([])                   # Z matrix of ELM
        self.b = 0
        self.threshold = threshold

    # Fixed Margin Algorithm    
    def FMP_algorithm(self, X, y, w_init, b_init, fixed_margin, idx, s):
        t = 0
        iterations = 0
        w = w_init
        b = b_init
        w_norm_1 = LA.norm(w, ord=1)
        last_t = -1
        lambda_t = 0
        alpha = np.zeros((X.shape[0]))
        while True:
            last_t = t
            e=0
            for k in range(0, y.shape[0]):
                i = int(idx[k])
                if(y[i]*(np.dot(X[i,:], w)+b) <= fixed_margin * w_norm_1 - self.lambda_param * alpha[i]):
                    if w_norm_1 != 0:
                        lambda_t = 1 - (self.eta*fixed_margin)/w_norm_1
                    else:
                        lambda_t = 1
                    alpha = alpha * lambda_t
                    alpha[i] = alpha[i] + self.eta    
                    w = w - self.eta * (fixed_margin * np.sign(w) - y[i] * X[i,:])
                    w_norm_1 = LA.norm(w, ord=1)
                    b = b + self.eta*y[i]
                    t += 1
                    e += 1
                    if k > s:
                        s += 1
                        j = s
                    else:
                        j=e
                    idx[k], idx[j] = idx[j], idx[k]
            iterations += 1
            if (t > self.max_updates or last_t == t):
                break
        if t<= self.max_updates:
            convergence=1
        else:
            convergence=0
        return w, b, convergence, t, iterations, idx, s

    # IMA Algorithm
    def IM_algorithm(self, X, y):
        self.w = np.zeros(self.H.shape[1])
        self.ws = [] 
        self.bs = [] 
        self.ws.append(self.w)
        self.bs.append(self.b)
        fixed_margin = 0#compute_margin(X, y, self.w_elm, self.b)
        t = 0
        convergence = 1
        updates=0
        iterations=0
        margin=[]
        margin.append(fixed_margin)
        idx = np.linspace(0, y.shape[0]-1, y.shape[0])
        s=0
        l=0
        while convergence==1 and t<self.IMA_iterations:
            w, b, convergence, updates_, iterations_, idx, s = self.FMP_algorithm(X, y, self.w, self.b, fixed_margin, idx, s)
            if convergence == 1:
                self.w = w
                self.b = b
                self.ws.append(self.w)
                self.bs.append(self.b)
            updates += updates_
            iterations += iterations_
            norm_w = LA.norm(w, ord=1)
            gamma1 = []
            gamma2 = []
            for i in range(0, y.shape[0]):
                if y[i] == 1:
                    gamma1.append((y[i]*(np.dot(X[i], self.w)+self.b))/norm_w)
                else:
                    gamma2.append((y[i]*(np.dot(X[i], self.w)+self.b))/norm_w)
            gamma1 = np.array(gamma1)
            gamma2 = np.array(gamma2)
            gamma1 = gamma1[gamma1>=0]
            gamma2 = gamma2[gamma2>=0]
            if len(gamma1) == 0:
                min_gamma1 = 0
            else:
                min_gamma1 = min(gamma1)
            if len(gamma2) == 0:
                min_gamma2 = 0
            else:
                min_gamma2 = min(gamma2)
            fixed_margin = max([(min_gamma1 + min_gamma2)/2, (1+self.delta_margin)*fixed_margin])
            #margin.append(compute_margin(X, y, self.w, self.b))
            t += 1
            l0_norm = 0
            remove = []
            for i in range(len(self.w)):
                if abs(self.w[i]) < self.threshold * self.w.max():
                    remove.append(i)
            self.w = np.delete(self.w, remove)
            self.Z = np.delete(self.Z, remove, axis=1)
            self.H = np.delete(self.H, remove, axis=1)
            X = np.delete(X, remove, axis=1)
        return t, updates, iterations

    # Function that manage the training of IMA ELM
    def fit(self, X, y, Z=[]):
        X_new = np.ones((X.shape[0], X.shape[1]+1))
        X_new[:,1:] = X
        X = X_new
        n = X.shape[1]
        if len(Z) == 0:
            self.Z = np.array([random.uniform(-0.5, 0.5) for i in range(n*self.n_neurons)]).reshape(n, self.n_neurons)
        else:
            self.Z = Z
        self.H = np.tanh(np.dot(X, self.Z))
        #w = np.dot(np.linalg.pinv(self.H), y)  
        #self.w_elm = w.reshape((w.shape[0],))
        iterations_IMA, updates, iterations = self.IM_algorithm(self.H, y) 
        return iterations_IMA, updates, iterations
            
    # Function to apply IMA ELM model
    def predict(self, X, use_IMA_w=True):
        X_new = np.ones((X.shape[0], X.shape[1]+1))
        X_new[:,1:] = X
        H = np.tanh(np.dot(X_new, self.Z))
        if use_IMA_w == True:
            y_predicted = np.sign(np.dot(H, self.w) + self.b)
        #else:
        #    y_predicted = np.sign(np.dot(H,  self.w_elm))
        y_predicted[y_predicted==0]=-1
        return y_predicted

## Commom ELM

In [7]:
class ELM(BaseEstimator, ClassifierMixin):
         
    def __init__(self, n_neurons):
        self.n_neurons = n_neurons
    
    def fit(self, X, y, Z=[]):
        X_new = np.ones((X.shape[0], X.shape[1]+1))
        X_new[:,1:] = X
        X = X_new
        n = X.shape[1]
        if len(Z) == 0:
            self.Z = np.array([random.uniform(-0.5, 0.5) for i in range(n*self.n_neurons)]).reshape(n, self.n_neurons)
        else:
            self.Z = Z
        self.H = np.tanh(np.dot(X, self.Z))
        self.w = np.dot(np.linalg.pinv(self.H), y)  
        return self.w, self.H, self.Z
            
    def predict(self, X):
        X_new = np.ones((X.shape[0], X.shape[1]+1))
        X_new[:,1:] = X
        H = np.tanh(np.dot(X_new, self.Z))
        y_predicted = np.sign(np.dot(H, self.w))
        y_predicted[y_predicted==0]=1
        return y_predicted

## Function to Capture Results

In [8]:
def grid_l(X, y, p):
    # GridSearch for lambda and learning rate of IMA ELM
    parameters = {'lambda_param':np.linspace(0.01, 10, 50)}
    clf = ELM_IMA_inf(n_neurons=p, delta_margin=10^-3, IMA_iterations=10, max_updates=10000)
    clf = GridSearchCV(clf, parameters, scoring='accuracy', cv=5, verbose=0)
    clf.fit(X, y)
    return clf.best_params_['lambda_param']

In [9]:
def results(X, y, n_splits, p, eta, IMA_iterations, lambda_param):      
    print(f'Parameters: p={p}, eta={eta}, lambda={lambda_param}')
    # Stratified k fold cross validation
    kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=72)
    i=0
    
    train_accuracy_IM_ELM_02 = np.zeros(n_splits)
    test_accuracy_IM_ELM_02 = np.zeros(n_splits)
    margin_IM_ELM_02 = np.zeros(n_splits)
    updates_02 = np.zeros(n_splits)
    iterations_FMP_02 = np.zeros(n_splits) 
    iterations_IMA_02 = np.zeros(n_splits)
    hidden_neurons_02 = np.zeros(n_splits)
    
    train_accuracy_IM_ELM_01 = np.zeros(n_splits)
    test_accuracy_IM_ELM_01 = np.zeros(n_splits)
    margin_IM_ELM_01 = np.zeros(n_splits)
    updates_01 = np.zeros(n_splits)
    iterations_FMP_01 = np.zeros(n_splits) 
    iterations_IMA_01 = np.zeros(n_splits)
    hidden_neurons_01 = np.zeros(n_splits)
    
    train_accuracy_IM_ELM_001 = np.zeros(n_splits)
    test_accuracy_IM_ELM_001 = np.zeros(n_splits)
    margin_IM_ELM_001 = np.zeros(n_splits)
    updates_001 = np.zeros(n_splits)
    iterations_FMP_001 = np.zeros(n_splits) 
    iterations_IMA_001 = np.zeros(n_splits)
    hidden_neurons_001 = np.zeros(n_splits)
    
    train_accuracy_IM_ELM_0001 = np.zeros(n_splits)
    test_accuracy_IM_ELM_0001 = np.zeros(n_splits)
    margin_IM_ELM_0001 = np.zeros(n_splits)
    updates_0001 = np.zeros(n_splits)
    iterations_FMP_0001 = np.zeros(n_splits) 
    iterations_IMA_0001 = np.zeros(n_splits)
    hidden_neurons_0001 = np.zeros(n_splits)
    
    train_accuracy_IM_ELM = np.zeros(n_splits)
    test_accuracy_IM_ELM = np.zeros(n_splits)
    margin_IM_ELM = np.zeros(n_splits)
    updates = np.zeros(n_splits)
    iterations_FMP = np.zeros(n_splits) 
    iterations_IMA = np.zeros(n_splits)
    hidden_neurons = np.zeros(n_splits)
    
    train_accuracy_ELM = np.zeros(n_splits)
    test_accuracy_ELM = np.zeros(n_splits)
    margin_ELM = np.zeros(n_splits)
        
    for train_index, test_index in kf.split(X, y):
        X_train = X[train_index,:]
        X_test = X[test_index,:]
        y_train = y[train_index]
        y_test = y[test_index]
               
        # ELM
        clf = ELM(n_neurons=p)
        clf.fit(X_train, y_train)
        y_hat=clf.predict(X_test)
        y_hat_train=clf.predict(X_train)
        margin_ELM[i] = compute_margin(clf.H[:,:], y_train, clf.w, 0)
        train_accuracy_ELM[i] = accuracy_score(y_train, y_hat_train)   
        test_accuracy_ELM[i] = accuracy_score(y_test, y_hat)
        Z = clf.Z
        
        # ELM-IMA 
        clf = ELM_IMA(n_neurons=p, eta=eta, lambda_param=lambda_param, delta_margin=10^-3,
                      IMA_iterations=IMA_iterations, max_updates=10000)
        iterations_IMA[i], updates[i], iterations_FMP[i] = clf.fit(X_train, y_train, Z)
        y_hat=clf.predict(X_test, use_IMA_w = True)
        y_hat_train=clf.predict(X_train, use_IMA_w = True)
        margin_IM_ELM[i] = compute_margin(clf.H[:,:], y_train, clf.w, clf.b)
        train_accuracy_IM_ELM[i] = accuracy_score(y_train, y_hat_train)   
        test_accuracy_IM_ELM[i] = accuracy_score(y_test, y_hat)
        hidden_neurons[i] = len(clf.w)  
        
        # ELM-IMA 0.2
        clf_02 = ELM_IMA_inf(n_neurons=p, eta=eta, lambda_param=lambda_param, delta_margin=10^-3,
                      IMA_iterations=IMA_iterations, max_updates=10000, threshold=0.2)
        iterations_IMA_02[i], updates_02[i], iterations_FMP_02[i] = clf_02.fit(X_train, y_train, Z)
        y_hat=clf_02.predict(X_test, use_IMA_w = True)
        y_hat_train=clf_02.predict(X_train, use_IMA_w = True)
        margin_IM_ELM_02[i] = compute_margin(clf_02.H[:,:], y_train, clf_02.w, clf_02.b)
        train_accuracy_IM_ELM_02[i] = accuracy_score(y_train, y_hat_train)   
        test_accuracy_IM_ELM_02[i] = accuracy_score(y_test, y_hat)
        hidden_neurons_02[i] = len(clf_02.w)
        
        # ELM-IMA 0.1
        clf_01 = ELM_IMA_inf(n_neurons=p, eta=eta, lambda_param=lambda_param, delta_margin=10^-3,
                      IMA_iterations=IMA_iterations, max_updates=10000, threshold=0.1)
        iterations_IMA_01[i], updates_01[i], iterations_FMP_01[i] = clf_01.fit(X_train, y_train, Z)
        y_hat=clf_01.predict(X_test, use_IMA_w = True)
        y_hat_train=clf_01.predict(X_train, use_IMA_w = True)
        margin_IM_ELM_01[i] = compute_margin(clf_01.H[:,:], y_train, clf_01.w, clf_01.b)
        train_accuracy_IM_ELM_01[i] = accuracy_score(y_train, y_hat_train)   
        test_accuracy_IM_ELM_01[i] = accuracy_score(y_test, y_hat)
        hidden_neurons_01[i] = len(clf_01.w)
        
                
        # ELM-IMA 0.01
        clf_001 = ELM_IMA_inf(n_neurons=p, eta=eta, lambda_param=lambda_param, delta_margin=10^-3,
                      IMA_iterations=IMA_iterations, max_updates=10000, threshold=0.01)
        iterations_IMA_001[i], updates_001[i], iterations_FMP_001[i] = clf_001.fit(X_train, y_train, Z)
        y_hat=clf_001.predict(X_test, use_IMA_w = True)
        y_hat_train=clf_001.predict(X_train, use_IMA_w = True)
        margin_IM_ELM_001[i] = compute_margin(clf_001.H[:,:], y_train, clf_001.w, clf_001.b)
        train_accuracy_IM_ELM_001[i] = accuracy_score(y_train, y_hat_train)   
        test_accuracy_IM_ELM_001[i] = accuracy_score(y_test, y_hat)
        hidden_neurons_001[i] = len(clf_001.w)

        # ELM-IMA 0.001
        clf_0001 = ELM_IMA_inf(n_neurons=p, eta=eta, lambda_param=lambda_param, delta_margin=10^-3,
                      IMA_iterations=IMA_iterations, max_updates=10000, threshold=0.001)
        iterations_IMA_0001[i], updates_0001[i], iterations_FMP_0001[i] = clf_0001.fit(X_train, y_train, Z)
        y_hat=clf_0001.predict(X_test, use_IMA_w = True)
        y_hat_train=clf_0001.predict(X_train, use_IMA_w = True)
        margin_IM_ELM_0001[i] = compute_margin(clf_0001.H[:,:], y_train, clf_0001.w, clf_0001.b)
        train_accuracy_IM_ELM_0001[i] = accuracy_score(y_train, y_hat_train)   
        test_accuracy_IM_ELM_0001[i] = accuracy_score(y_test, y_hat)
        hidden_neurons_0001[i] = len(clf_0001.w)  
        i+=1
    
    print("********* Results ELM-IMA 0.2**************")
    print("Acc train: " + '{:.4f}'.format(train_accuracy_IM_ELM_02.mean())+ "+/-" + '{:.4f}'.format(train_accuracy_IM_ELM_02.std()))
    print("Acc test: " + '{:.4f}'.format(test_accuracy_IM_ELM_02.mean()) + "+/-" + '{:.4f}'.format(test_accuracy_IM_ELM_02.std()))
    print("Iterations: " + '{:.4f}'.format(iterations_FMP_02.mean())+ "+/-" + '{:.4f}'.format(iterations_FMP_02.std()))
    print("Iterations IMA: " + '{:.4f}'.format(iterations_IMA_02.mean())+ "+/-" + '{:.4f}'.format(iterations_IMA_02.std()))
    print("Updates: " + '{:.4f}'.format(updates_02.mean())+ "+/-" + '{:.4f}'.format(updates_02.std()))
    print("Margin: " + '{:.9f}'.format(margin_IM_ELM_02.mean())+ "+/-" + '{:.9f}'.format(margin_IM_ELM_02.std()))
    print("Hidden Neurons: " + '{:.2f}'.format(hidden_neurons_02.mean()) + "+/-" + '{:.2f}'.format(hidden_neurons_02.std()))
    
    print("********* Results ELM-IMA 0.1**************")
    print("Acc train: " + '{:.4f}'.format(train_accuracy_IM_ELM_01.mean())+ "+/-" + '{:.4f}'.format(train_accuracy_IM_ELM_01.std()))
    print("Acc test: " + '{:.4f}'.format(test_accuracy_IM_ELM_01.mean()) + "+/-" + '{:.4f}'.format(test_accuracy_IM_ELM_01.std()))
    print("Iterations: " + '{:.4f}'.format(iterations_FMP_01.mean())+ "+/-" + '{:.4f}'.format(iterations_FMP_01.std()))
    print("Iterations IMA: " + '{:.4f}'.format(iterations_IMA_01.mean())+ "+/-" + '{:.4f}'.format(iterations_IMA_01.std()))
    print("Updates: " + '{:.4f}'.format(updates_01.mean())+ "+/-" + '{:.4f}'.format(updates_01.std()))
    print("Margin: " + '{:.9f}'.format(margin_IM_ELM_01.mean())+ "+/-" + '{:.9f}'.format(margin_IM_ELM_01.std()))
    print("Hidden Neurons: " + '{:.2f}'.format(hidden_neurons_01.mean()) + "+/-" + '{:.2f}'.format(hidden_neurons_01.std()))
    
    print("********* Results ELM-IMA 0.01**************")
    print("Acc train: " + '{:.4f}'.format(train_accuracy_IM_ELM_001.mean())+ "+/-" + '{:.4f}'.format(train_accuracy_IM_ELM_001.std()))
    print("Acc test: " + '{:.4f}'.format(test_accuracy_IM_ELM_001.mean()) + "+/-" + '{:.4f}'.format(test_accuracy_IM_ELM_001.std()))
    print("Iterations: " + '{:.4f}'.format(iterations_FMP_001.mean())+ "+/-" + '{:.4f}'.format(iterations_FMP_001.std()))
    print("Iterations IMA: " + '{:.4f}'.format(iterations_IMA_001.mean())+ "+/-" + '{:.4f}'.format(iterations_IMA_001.std()))
    print("Updates: " + '{:.4f}'.format(updates_001.mean())+ "+/-" + '{:.4f}'.format(updates_001.std()))
    print("Margin: " + '{:.9f}'.format(margin_IM_ELM_001.mean())+ "+/-" + '{:.9f}'.format(margin_IM_ELM_001.std()))
    print("Hidden Neurons: " + '{:.2f}'.format(hidden_neurons_001.mean()) + "+/-" + '{:.2f}'.format(hidden_neurons_001.std()))
    
    print("********* Results ELM-IMA 0.001**************")
    print("Acc train: " + '{:.4f}'.format(train_accuracy_IM_ELM_0001.mean())+ "+/-" + '{:.4f}'.format(train_accuracy_IM_ELM_0001.std()))
    print("Acc test: " + '{:.4f}'.format(test_accuracy_IM_ELM_0001.mean()) + "+/-" + '{:.4f}'.format(test_accuracy_IM_ELM_0001.std()))
    print("Iterations: " + '{:.4f}'.format(iterations_FMP_0001.mean())+ "+/-" + '{:.4f}'.format(iterations_FMP_0001.std()))
    print("Iterations IMA: " + '{:.4f}'.format(iterations_IMA_0001.mean())+ "+/-" + '{:.4f}'.format(iterations_IMA_0001.std()))
    print("Updates: " + '{:.4f}'.format(updates_0001.mean())+ "+/-" + '{:.4f}'.format(updates_0001.std()))
    print("Margin: " + '{:.9f}'.format(margin_IM_ELM_0001.mean())+ "+/-" + '{:.9f}'.format(margin_IM_ELM_0001.std()))
    print("Hidden Neurons: " + '{:.2f}'.format(hidden_neurons_0001.mean()) + "+/-" + '{:.2f}'.format(hidden_neurons_0001.std()))
    
    print("********* Results ELM-IMA p = 2**************")
    print("Acc train: " + '{:.4f}'.format(train_accuracy_IM_ELM.mean())+ "+/-" + '{:.4f}'.format(train_accuracy_IM_ELM.std()))
    print("Acc test: " + '{:.4f}'.format(test_accuracy_IM_ELM.mean()) + "+/-" + '{:.4f}'.format(test_accuracy_IM_ELM.std()))
    print("Iterations: " + '{:.4f}'.format(iterations_FMP.mean())+ "+/-" + '{:.4f}'.format(iterations_FMP.std()))
    print("Iterations IMA: " + '{:.4f}'.format(iterations_IMA.mean())+ "+/-" + '{:.4f}'.format(iterations_IMA.std()))
    print("Updates: " + '{:.4f}'.format(updates.mean())+ "+/-" + '{:.4f}'.format(updates.std()))
    print("Margin: " + '{:.9f}'.format(margin_IM_ELM.mean())+ "+/-" + '{:.9f}'.format(margin_IM_ELM.std()))
    print("Hidden Neurons: " + '{:.2f}'.format(hidden_neurons.mean()) + "+/-" + '{:.2f}'.format(hidden_neurons.std()))
            
    print("********* Results ELM **************")
    print("Acc train: " + '{:.4f}'.format(train_accuracy_ELM.mean())+ "+/-" + '{:.4f}'.format(train_accuracy_ELM.std()))
    print("Acc test: " + '{:.4f}'.format(test_accuracy_ELM.mean()) + "+/-" + '{:.4f}'.format(test_accuracy_ELM.std()))
    print("Margin: " + '{:.9f}'.format(margin_ELM.mean())+  "+/-" + '{:.9f}'.format(margin_ELM.std()))  

In [10]:
def run(X,y):
    # Normalizing data:
    normalizer = MinMaxScaler()
    X = normalizer.fit_transform(X)
    n = len(X)
    if n>1000:
        n=1000
    for p in [n]:
        l = grid_l(X, y, p=p)
        print(f"Experimento com {p} neurônios:" )
        results(X, y, n_splits=10, p=p, eta=0.1, IMA_iterations=20, lambda_param=l)
        print("\n\n")

## Application on Iris Dataset


In [10]:
iris = datasets.load_iris()
X = iris.data
# setosa - 0, versicolor - 1, virginica - 2  
y = iris.target 
# O problema agora possui apenas as classes y=-1 e y=1
y[y>0] = 1
y[y==0] = -1
run(X, y)

Experimento com 150 neurônios:
Parameters: p=150, eta=0.1, lambda=0.01
********* Results ELM-IMA 0.2**************
Acc train: 1.0000+/-0.0000
Acc test: 1.0000+/-0.0000
Iterations: 254.1000+/-116.7326
Iterations IMA: 20.0000+/-0.0000
Updates: 629.1000+/-384.5279
Margin: 0.257552670+/-0.064084934
Hidden Neurons: 6.90+/-4.18
********* Results ELM-IMA 0.1**************
Acc train: 1.0000+/-0.0000
Acc test: 1.0000+/-0.0000
Iterations: 211.8000+/-59.5664
Iterations IMA: 20.0000+/-0.0000
Updates: 503.2000+/-183.4627
Margin: 0.306136142+/-0.048679447
Hidden Neurons: 10.30+/-4.43
********* Results ELM-IMA 0.01**************
Acc train: 1.0000+/-0.0000
Acc test: 1.0000+/-0.0000
Iterations: 314.1000+/-97.8933
Iterations IMA: 20.0000+/-0.0000
Updates: 798.9000+/-264.5848
Margin: 0.335008497+/-0.052644213
Hidden Neurons: 26.70+/-14.24
********* Results ELM-IMA 0.001**************
Acc train: 1.0000+/-0.0000
Acc test: 1.0000+/-0.0000
Iterations: 367.5000+/-216.1316
Iterations IMA: 20.0000+/-0.0000
Upda

## Application on Synthetic Dataset

In [11]:
synthetic_dataset = pd.read_csv('data/synthetic_dataset/synthetic_control.data', sep="\s+",  header=None, engine='python')
X = synthetic_dataset.to_numpy()
y = np.concatenate((np.ones(100), np.ones(200)*-1, np.ones(100), np.ones(100)*-1,np.ones(100)))
run(X, y)

Experimento com 600 neurônios:
Parameters: p=600, eta=0.1, lambda=0.8255102040816327
********* Results ELM-IMA 0.2**************
Acc train: 1.0000+/-0.0000
Acc test: 0.9917+/-0.0083
Iterations: 122.2000+/-15.6895
Iterations IMA: 20.0000+/-0.0000
Updates: 682.3000+/-135.3817
Margin: 0.260410781+/-0.049472923
Hidden Neurons: 93.30+/-9.63
********* Results ELM-IMA 0.1**************
Acc train: 1.0000+/-0.0000
Acc test: 0.9933+/-0.0082
Iterations: 136.8000+/-30.5084
Iterations IMA: 20.0000+/-0.0000
Updates: 952.2000+/-144.6207
Margin: 0.293958173+/-0.043308477
Hidden Neurons: 161.00+/-15.30
********* Results ELM-IMA 0.01**************
Acc train: 1.0000+/-0.0000
Acc test: 0.9950+/-0.0076
Iterations: 163.5000+/-29.9775
Iterations IMA: 20.0000+/-0.0000
Updates: 1316.1000+/-165.3913
Margin: 0.336457117+/-0.023427526
Hidden Neurons: 284.10+/-32.49
********* Results ELM-IMA 0.001**************
Acc train: 1.0000+/-0.0000
Acc test: 0.9933+/-0.0082
Iterations: 168.9000+/-25.3947
Iterations IMA: 20.0

## Application on Robot  Dataset



In [12]:
robot_dataset = pd.read_csv('data/robot/lp4_data.csv', delimiter =',')
X = robot_dataset.to_numpy().reshape([117,90])
y = np.concatenate((np.ones(24), np.ones(117-24)*-1))
run(X, y)

Experimento com 117 neurônios:
Parameters: p=117, eta=0.1, lambda=3.2720408163265304
********* Results ELM-IMA 0.2**************
Acc train: 0.8081+/-0.0390
Acc test: 0.8045+/-0.0507
Iterations: 367.8000+/-113.6634
Iterations IMA: 12.1000+/-3.7802
Updates: 10396.3000+/-3025.4598
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 8.30+/-8.23
********* Results ELM-IMA 0.1**************
Acc train: 0.7949+/-0.0043
Acc test: 0.7788+/-0.0737
Iterations: 440.8000+/-317.0630
Iterations IMA: 10.6000+/-6.1351
Updates: 9759.4000+/-3959.2896
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 24.50+/-17.09
********* Results ELM-IMA 0.01**************
Acc train: 0.7949+/-0.0043
Acc test: 0.7955+/-0.0376
Iterations: 373.7000+/-69.3600
Iterations IMA: 8.2000+/-4.6433
Updates: 12334.7000+/-968.1598
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 76.00+/-16.47
********* Results ELM-IMA 0.001**************
Acc train: 0.7949+/-0.0043
Acc test: 0.7955+/-0.0376
Iterations: 347.8000+/-64.5752
Iterations IMA: 

## Application on Mushroom Dataset

In [13]:
df = pd.read_csv('data/Mushroom/agaricus-lepiota.data', delimiter =',', header=None)
df = df.replace("?", np.nan) 
df = df.dropna() 
y = df[0].to_numpy()
X = df.drop([0], axis='columns')
X = pd.get_dummies(X).to_numpy()
y[np.where(y=='e')] = -1
y[np.where(y=='p')] = 1
y = np.array(y.tolist())
run(X, y)

Experimento com 1000 neurônios:
Parameters: p=1000, eta=0.1, lambda=5.106938775510204
********* Results ELM-IMA 0.2**************
Acc train: 1.0000+/-0.0000
Acc test: 1.0000+/-0.0000
Iterations: 91.5000+/-7.9152
Iterations IMA: 20.0000+/-0.0000
Updates: 430.2000+/-95.7286
Margin: 0.917358618+/-0.053815506
Hidden Neurons: 92.50+/-13.20
********* Results ELM-IMA 0.1**************
Acc train: 1.0000+/-0.0000
Acc test: 1.0000+/-0.0000
Iterations: 91.4000+/-7.8892
Iterations IMA: 20.0000+/-0.0000
Updates: 471.9000+/-61.0253
Margin: 1.090691926+/-0.067669721
Hidden Neurons: 159.60+/-25.18
********* Results ELM-IMA 0.01**************
Acc train: 1.0000+/-0.0000
Acc test: 1.0000+/-0.0000
Iterations: 96.2000+/-9.3038
Iterations IMA: 20.0000+/-0.0000
Updates: 618.2000+/-123.0746
Margin: 1.272439453+/-0.053223674
Hidden Neurons: 285.30+/-34.06
********* Results ELM-IMA 0.001**************
Acc train: 1.0000+/-0.0000
Acc test: 1.0000+/-0.0000
Iterations: 120.6000+/-13.5588
Iterations IMA: 20.0000+/-0

## Application on Ionosphere Dataset

In [14]:
ionosphere_dataset = pd.read_csv('data/Ionosphere/ionosphere.data', names=list(range(0,35)), sep=',')
y = ionosphere_dataset[34].to_numpy()
X = ionosphere_dataset.drop([34], axis='columns').to_numpy()
y[np.where(y=='g')] = 1
y[np.where(y=='b')] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 351 neurônios:
Parameters: p=351, eta=0.1, lambda=5.922448979591836
********* Results ELM-IMA 0.2**************
Acc train: 0.9642+/-0.0298
Acc test: 0.8860+/-0.0711
Iterations: 199.5000+/-19.8809
Iterations IMA: 20.0000+/-0.0000
Updates: 2535.6000+/-332.6266
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 109.70+/-15.52
********* Results ELM-IMA 0.1**************
Acc train: 0.9810+/-0.0066
Acc test: 0.9002+/-0.0617
Iterations: 233.9000+/-23.8053
Iterations IMA: 20.0000+/-0.0000
Updates: 3129.8000+/-386.9470
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 170.40+/-16.95
********* Results ELM-IMA 0.01**************
Acc train: 0.9820+/-0.0057
Acc test: 0.8945+/-0.0573
Iterations: 286.7000+/-26.4766
Iterations IMA: 20.0000+/-0.0000
Updates: 3907.6000+/-565.6473
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 267.80+/-13.30
********* Results ELM-IMA 0.001**************
Acc train: 0.9851+/-0.0049
Acc test: 0.9144+/-0.0586
Iterations: 311.2000+/-34.6895
Iterations IMA: 2

## Application on Banknote Dataset

In [15]:
# read in banknote authentication set
banknotes = pd.read_csv('data/banknote/data_banknote_authentication.txt', names=['variance', 'skewness', 'curtosis', 'entropy', 'class'])
# convert to array
X = banknotes[['variance', 'skewness', 'curtosis', 'entropy']].to_numpy()
y = banknotes[['class']].to_numpy()
y[np.where(y==0)] = -1
run(X, y)

Experimento com 1000 neurônios:
Parameters: p=1000, eta=0.1, lambda=0.01
********* Results ELM-IMA 0.2**************
Acc train: 0.9956+/-0.0118
Acc test: 0.9964+/-0.0088
Iterations: 1783.3000+/-577.7217
Iterations IMA: 20.0000+/-0.0000
Updates: 21832.1000+/-5930.9480
Margin: 0.009881152+/-0.010656475
Hidden Neurons: 185.00+/-29.28
********* Results ELM-IMA 0.1**************
Acc train: 0.9998+/-0.0007
Acc test: 1.0000+/-0.0000
Iterations: 2259.2000+/-442.0119
Iterations IMA: 20.0000+/-0.0000
Updates: 29857.0000+/-4395.3984
Margin: 0.018204365+/-0.011994846
Hidden Neurons: 323.00+/-32.44
********* Results ELM-IMA 0.01**************
Acc train: 0.9996+/-0.0012
Acc test: 1.0000+/-0.0000
Iterations: 2552.3000+/-835.9853
Iterations IMA: 18.2000+/-5.4000
Updates: 35224.2000+/-8681.7558
Margin: 0.029945679+/-0.010564932
Hidden Neurons: 571.80+/-144.69
********* Results ELM-IMA 0.001**************
Acc train: 1.0000+/-0.0000
Acc test: 1.0000+/-0.0000
Iterations: 3132.5000+/-696.3636
Iterations IM

## Application on Wine Dataset

In [16]:
wine_dataset = pd.read_csv('data/wine/wine.data', names=['Class', 'Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium', 'Total phenols', 'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 'OD280/OD315', 'Proline'])
# convert to array
y = wine_dataset[['Class']].to_numpy()
X = wine_dataset.drop("Class",axis='columns').to_numpy()
y[np.where(y==3)] = 1
y[np.where(y==2)] = -1
run(X, y)

Experimento com 178 neurônios:
Parameters: p=178, eta=0.1, lambda=0.8255102040816327
********* Results ELM-IMA 0.2**************
Acc train: 0.9981+/-0.0040
Acc test: 0.9775+/-0.0276
Iterations: 106.1000+/-11.4843
Iterations IMA: 20.0000+/-0.0000
Updates: 381.0000+/-47.5289
Margin: 0.055402287+/-0.034348756
Hidden Neurons: 53.10+/-6.92
********* Results ELM-IMA 0.1**************
Acc train: 1.0000+/-0.0000
Acc test: 0.9663+/-0.0275
Iterations: 108.5000+/-12.6669
Iterations IMA: 20.0000+/-0.0000
Updates: 442.0000+/-52.4729
Margin: 0.078180273+/-0.022075788
Hidden Neurons: 79.80+/-7.77
********* Results ELM-IMA 0.01**************
Acc train: 1.0000+/-0.0000
Acc test: 0.9663+/-0.0275
Iterations: 125.6000+/-15.6665
Iterations IMA: 20.0000+/-0.0000
Updates: 592.3000+/-64.3227
Margin: 0.079149777+/-0.019679419
Hidden Neurons: 119.40+/-6.65
********* Results ELM-IMA 0.001**************
Acc train: 1.0000+/-0.0000
Acc test: 0.9719+/-0.0281
Iterations: 131.7000+/-10.9366
Iterations IMA: 20.0000+/-0

## Application on WDBC Dataset

In [17]:
wdbc_dataset = pd.read_csv('data/WDBC/wdbc.data', names=list(range(0,32)))
# convert to array
y = wdbc_dataset[1].to_numpy()
X = wdbc_dataset.drop([0, 1],axis='columns').to_numpy()
y[np.where(y=='B')] = 1
y[np.where(y=='M')] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 569 neurônios:
Parameters: p=569, eta=0.1, lambda=6.1263265306122445
********* Results ELM-IMA 0.2**************
Acc train: 0.9836+/-0.0051
Acc test: 0.9596+/-0.0222
Iterations: 162.9000+/-21.6400
Iterations IMA: 20.0000+/-0.0000
Updates: 1440.5000+/-150.7383
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 111.80+/-14.97
********* Results ELM-IMA 0.1**************
Acc train: 0.9852+/-0.0045
Acc test: 0.9719+/-0.0195
Iterations: 203.4000+/-16.1258
Iterations IMA: 20.0000+/-0.0000
Updates: 1985.9000+/-222.0705
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 171.20+/-28.51
********* Results ELM-IMA 0.01**************
Acc train: 0.9850+/-0.0057
Acc test: 0.9666+/-0.0228
Iterations: 273.9000+/-21.4963
Iterations IMA: 20.0000+/-0.0000
Updates: 3127.1000+/-331.4995
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 268.50+/-36.21
********* Results ELM-IMA 0.001**************
Acc train: 0.9859+/-0.0026
Acc test: 0.9684+/-0.0204
Iterations: 360.7000+/-37.8710
Iterations IMA: 

## Application on Sonar Dataset

In [18]:
sonar_dataset = pd.read_csv('data/sonar/sonar.all-data', names=list(range(0,61)), sep=',')
y = sonar_dataset[60].to_numpy()
X = sonar_dataset.drop([60], axis='columns').to_numpy()
y[np.where(y=='R')] = 1
y[np.where(y=='M')] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 208 neurônios:
Parameters: p=208, eta=0.1, lambda=3.4759183673469387
********* Results ELM-IMA 0.2**************
Acc train: 0.9904+/-0.0106
Acc test: 0.7914+/-0.1048
Iterations: 173.7000+/-25.6361
Iterations IMA: 20.0000+/-0.0000
Updates: 1882.5000+/-382.1458
Margin: 0.003311149+/-0.005566764
Hidden Neurons: 65.40+/-13.29
********* Results ELM-IMA 0.1**************
Acc train: 0.9952+/-0.0074
Acc test: 0.8252+/-0.1108
Iterations: 196.3000+/-32.7416
Iterations IMA: 20.0000+/-0.0000
Updates: 2309.6000+/-489.3927
Margin: 0.013545584+/-0.014041514
Hidden Neurons: 103.30+/-10.62
********* Results ELM-IMA 0.01**************
Acc train: 0.9995+/-0.0016
Acc test: 0.8205+/-0.0989
Iterations: 223.6000+/-31.3662
Iterations IMA: 20.0000+/-0.0000
Updates: 3071.9000+/-674.1991
Margin: 0.014426006+/-0.012344728
Hidden Neurons: 162.40+/-4.59
********* Results ELM-IMA 0.001**************
Acc train: 1.0000+/-0.0000
Acc test: 0.8207+/-0.1077
Iterations: 229.6000+/-37.2215
Iterations IMA: 20

## Application on Pima Dataset

In [19]:
pima_dataset = pd.read_csv('data/diabetes/diabetes.csv', sep=",", engine='python')
y = pima_dataset['Outcome'].to_numpy()
X = pima_dataset.drop(['Outcome'], axis='columns').to_numpy()
y[np.where(y==0)] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 768 neurônios:
Parameters: p=768, eta=0.1, lambda=4.903061224489796
********* Results ELM-IMA 0.2**************
Acc train: 0.7636+/-0.0521
Acc test: 0.7475+/-0.0398
Iterations: 253.1000+/-12.0702
Iterations IMA: 20.0000+/-0.0000
Updates: 22208.5000+/-1802.8153
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 94.60+/-15.14
********* Results ELM-IMA 0.1**************
Acc train: 0.7818+/-0.0155
Acc test: 0.7670+/-0.0423
Iterations: 292.7000+/-15.2908
Iterations IMA: 20.0000+/-0.0000
Updates: 29186.9000+/-2208.9733
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 162.90+/-25.32
********* Results ELM-IMA 0.01**************
Acc train: 0.7839+/-0.0148
Acc test: 0.7551+/-0.0431
Iterations: 395.3000+/-22.6276
Iterations IMA: 20.0000+/-0.0000
Updates: 46159.2000+/-3880.9585
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 386.10+/-75.07
********* Results ELM-IMA 0.001**************
Acc train: 0.7866+/-0.0089
Acc test: 0.7617+/-0.0264
Iterations: 454.6000+/-23.4742
Iterations I

## Application on Statlog Dataset

In [20]:
statlog_dataset = pd.read_csv('data/statlog/heart.dat', sep=" ", header=None, engine='python')
y = statlog_dataset[13].to_numpy()
X = statlog_dataset.drop([13], axis='columns').to_numpy()
y[np.where(y==2)] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 270 neurônios:
Parameters: p=270, eta=0.1, lambda=7.75734693877551
********* Results ELM-IMA 0.2**************
Acc train: 0.8716+/-0.0125
Acc test: 0.8333+/-0.0647
Iterations: 165.5000+/-12.5479
Iterations IMA: 20.0000+/-0.0000
Updates: 2824.8000+/-350.1139
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 32.90+/-10.17
********* Results ELM-IMA 0.1**************
Acc train: 0.8778+/-0.0122
Acc test: 0.8333+/-0.0781
Iterations: 190.6000+/-15.6538
Iterations IMA: 20.0000+/-0.0000
Updates: 3510.0000+/-416.0466
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 56.60+/-12.18
********* Results ELM-IMA 0.01**************
Acc train: 0.8877+/-0.0121
Acc test: 0.8148+/-0.0811
Iterations: 251.2000+/-17.0634
Iterations IMA: 20.0000+/-0.0000
Updates: 5479.1000+/-475.0797
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 134.40+/-22.14
********* Results ELM-IMA 0.001**************
Acc train: 0.8885+/-0.0150
Acc test: 0.8296+/-0.0848
Iterations: 291.4000+/-18.9536
Iterations IMA: 20.0

## Application on Mammographic Dataset

In [21]:
mammo = pd.read_csv('data/mammographic/mammographic_masses.data', sep=",", header=None, engine='python')
mammo = mammo.replace("?", np.nan)
mammo = mammo.dropna()
y = mammo[5].to_numpy()
X = mammo.drop([5], axis='columns').to_numpy()
y[np.where(y==0)] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 830 neurônios:
Parameters: p=830, eta=0.1, lambda=4.0875510204081635
********* Results ELM-IMA 0.2**************
Acc train: 0.8139+/-0.0285
Acc test: 0.8193+/-0.0598
Iterations: 436.2000+/-38.1885
Iterations IMA: 20.0000+/-0.0000
Updates: 37954.1000+/-4053.4353
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 261.10+/-47.97
********* Results ELM-IMA 0.1**************
Acc train: 0.8254+/-0.0151
Acc test: 0.8084+/-0.0501
Iterations: 491.2000+/-34.9737
Iterations IMA: 20.0000+/-0.0000
Updates: 46917.4000+/-3062.6736
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 382.20+/-52.46
********* Results ELM-IMA 0.01**************
Acc train: 0.8311+/-0.0074
Acc test: 0.8253+/-0.0497
Iterations: 594.7000+/-37.2318
Iterations IMA: 20.0000+/-0.0000
Updates: 61293.8000+/-3194.6830
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 541.70+/-43.58
********* Results ELM-IMA 0.001**************
Acc train: 0.8303+/-0.0053
Acc test: 0.8289+/-0.0453
Iterations: 659.5000+/-33.9094
Iterations

## Application on Haberman Dataset

In [22]:
haberman = pd.read_csv('data/haberman/haberman.data', sep=",", header=None, engine='python')
y = haberman[3].to_numpy()
X = haberman.drop([3], axis='columns').to_numpy()
y[np.where(y==2)] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 306 neurônios:
Parameters: p=306, eta=0.1, lambda=0.8255102040816327
********* Results ELM-IMA 0.2**************
Acc train: 0.7167+/-0.0547
Acc test: 0.7256+/-0.0378
Iterations: 241.1000+/-63.4704
Iterations IMA: 9.0000+/-4.1231
Updates: 15708.5000+/-2029.9437
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 11.30+/-5.51
********* Results ELM-IMA 0.1**************
Acc train: 0.7346+/-0.0027
Acc test: 0.7319+/-0.0203
Iterations: 254.0000+/-98.5870
Iterations IMA: 8.1000+/-4.7212
Updates: 16704.0000+/-2740.6056
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 47.10+/-33.32
********* Results ELM-IMA 0.01**************
Acc train: 0.7349+/-0.0067
Acc test: 0.7288+/-0.0194
Iterations: 428.7000+/-197.0883
Iterations IMA: 12.4000+/-6.4374
Updates: 29573.4000+/-8681.5656
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 181.60+/-76.38
********* Results ELM-IMA 0.001**************
Acc train: 0.7375+/-0.0072
Acc test: 0.7353+/-0.0225
Iterations: 433.4000+/-272.3080
Iterations IM

## Application on Transfusion Dataset

In [23]:
transfusion = pd.read_csv('data/transfusion/transfusion.data', sep=",", engine='python')
y = transfusion["whether he/she donated blood in March 2007"].to_numpy()
X = transfusion.drop(["whether he/she donated blood in March 2007"], axis='columns').to_numpy()
y[np.where(y==0)] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 748 neurônios:
Parameters: p=748, eta=0.1, lambda=0.6216326530612245
********* Results ELM-IMA 0.2**************
Acc train: 0.7469+/-0.0708
Acc test: 0.7475+/-0.0916
Iterations: 381.3000+/-133.4519
Iterations IMA: 13.4000+/-6.3906
Updates: 29069.4000+/-4924.1466
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 9.40+/-6.71
********* Results ELM-IMA 0.1**************
Acc train: 0.7748+/-0.0062
Acc test: 0.7714+/-0.0187
Iterations: 480.4000+/-170.8825
Iterations IMA: 15.4000+/-6.2161
Updates: 40882.9000+/-10284.9809
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 24.60+/-22.87
********* Results ELM-IMA 0.01**************
Acc train: 0.7803+/-0.0129
Acc test: 0.7808+/-0.0211
Iterations: 340.5000+/-285.0969
Iterations IMA: 9.6000+/-8.5112
Updates: 48675.2000+/-32873.2939
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 419.50+/-257.23
********* Results ELM-IMA 0.001**************
Acc train: 0.7782+/-0.0097
Acc test: 0.7702+/-0.0265
Iterations: 299.1000+/-248.8990
Iteratio

## Application on Australian Credit 

In [24]:
australian = pd.read_csv('data/australian_credit/australian.dat', header=None, sep=" ", engine='python')
australian = australian.replace("?", np.nan)
australian = australian.dropna()
y = australian[14].to_numpy()
X = australian.drop([14], axis='columns').to_numpy()
y[np.where(y==0)] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 690 neurônios:
Parameters: p=690, eta=0.1, lambda=9.592244897959183
********* Results ELM-IMA 0.2**************
Acc train: 0.8853+/-0.0223
Acc test: 0.8493+/-0.0244
Iterations: 253.6000+/-24.9808
Iterations IMA: 20.0000+/-0.0000
Updates: 9142.5000+/-1274.9587
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 93.00+/-31.25
********* Results ELM-IMA 0.1**************
Acc train: 0.8940+/-0.0080
Acc test: 0.8565+/-0.0271
Iterations: 305.3000+/-29.8732
Iterations IMA: 20.0000+/-0.0000
Updates: 12370.5000+/-1992.6548
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 179.00+/-57.70
********* Results ELM-IMA 0.01**************
Acc train: 0.8953+/-0.0065
Acc test: 0.8565+/-0.0314
Iterations: 418.7000+/-13.6092
Iterations IMA: 20.0000+/-0.0000
Updates: 20112.1000+/-1453.0185
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 399.80+/-69.64
********* Results ELM-IMA 0.001**************
Acc train: 0.9014+/-0.0061
Acc test: 0.8565+/-0.0340
Iterations: 462.0000+/-13.7332
Iterations IM

## Application on Breast Cancer Dataset

In [25]:
breast = pd.read_csv('data/breast/breast.data', header=None, sep=",", engine='python')
breast = breast.replace("?", np.nan)
breast = breast.dropna()
y = breast[10].to_numpy()
X = breast.drop([0, 10], axis='columns').to_numpy()
y[np.where(y==4)] = 1
y[np.where(y==2)] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 683 neurônios:
Parameters: p=683, eta=0.1, lambda=8.77673469387755
********* Results ELM-IMA 0.2**************
Acc train: 0.9756+/-0.0029
Acc test: 0.9692+/-0.0167
Iterations: 110.1000+/-4.7000
Iterations IMA: 20.0000+/-0.0000
Updates: 1197.1000+/-129.1917
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 13.20+/-7.22
********* Results ELM-IMA 0.1**************
Acc train: 0.9753+/-0.0032
Acc test: 0.9692+/-0.0191
Iterations: 119.7000+/-10.0305
Iterations IMA: 20.0000+/-0.0000
Updates: 1388.1000+/-134.5039
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 18.90+/-7.89
********* Results ELM-IMA 0.01**************
Acc train: 0.9746+/-0.0018
Acc test: 0.9707+/-0.0197
Iterations: 188.0000+/-18.1769
Iterations IMA: 20.0000+/-0.0000
Updates: 2593.4000+/-358.7863
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 97.30+/-34.81
********* Results ELM-IMA 0.001**************
Acc train: 0.9732+/-0.0026
Acc test: 0.9692+/-0.0180
Iterations: 257.2000+/-12.1062
Iterations IMA: 20.0000+

## Application on Spam Dataset

In [26]:
spam = pd.read_csv('data/spam/spambase.data', header=None, sep=",", engine='python')
y = spam[57].to_numpy()
X = spam.drop([57], axis='columns').to_numpy()
y[np.where(y==0)] = -1
y = np.array(y.tolist())
run(X, y)

Experimento com 1000 neurônios:
Parameters: p=1000, eta=0.1, lambda=5.922448979591836
********* Results ELM-IMA 0.2**************
Acc train: 0.9160+/-0.0157
Acc test: 0.9107+/-0.0206
Iterations: 307.2000+/-35.3717
Iterations IMA: 20.0000+/-0.0000
Updates: 44785.9000+/-2029.9652
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 188.20+/-26.19
********* Results ELM-IMA 0.1**************
Acc train: 0.9272+/-0.0088
Acc test: 0.9259+/-0.0109
Iterations: 358.1000+/-22.1244
Iterations IMA: 20.0000+/-0.0000
Updates: 62303.0000+/-3721.7664
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 380.20+/-19.61
********* Results ELM-IMA 0.01**************
Acc train: 0.9223+/-0.0109
Acc test: 0.9113+/-0.0196
Iterations: 478.0000+/-28.1709
Iterations IMA: 20.0000+/-0.0000
Updates: 93182.1000+/-5274.0501
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 752.10+/-17.14
********* Results ELM-IMA 0.001**************
Acc train: 0.9274+/-0.0077
Acc test: 0.9185+/-0.0115
Iterations: 499.5000+/-32.6963
Iteration

## Application on Glass Dataset

In [27]:
# Loading dataset:
headers = ["Id", "RI", "Na", "Mg", "Al", "Si", "K", "Ca", "Ba", "Fe", "Class"]
df = pd.read_csv("~/Documents/UFMG/Graduation/10/Reconhecimento de padrões/list/pattern-recognition-exercises/list_5/databases/glass.csv", names = headers)
X = df.drop("Class", axis=1)
X = X.drop("Id", axis=1)
y = df["Class"]
X = X.to_numpy()
y = y.to_numpy()
y[np.where(y>1)] = -1
run(X, y)

Experimento com 214 neurônios:
Parameters: p=214, eta=0.1, lambda=7.75734693877551
********* Results ELM-IMA 0.2**************
Acc train: 0.7606+/-0.0327
Acc test: 0.7571+/-0.1178
Iterations: 153.4000+/-19.6326
Iterations IMA: 20.0000+/-0.0000
Updates: 2186.1000+/-125.6101
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 26.30+/-6.65
********* Results ELM-IMA 0.1**************
Acc train: 0.7565+/-0.0215
Acc test: 0.7429+/-0.1153
Iterations: 172.0000+/-24.1785
Iterations IMA: 20.0000+/-0.0000
Updates: 2377.6000+/-119.2478
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 41.20+/-9.93
********* Results ELM-IMA 0.01**************
Acc train: 0.7596+/-0.0307
Acc test: 0.7377+/-0.1146
Iterations: 205.3000+/-25.1478
Iterations IMA: 20.0000+/-0.0000
Updates: 3214.4000+/-238.3175
Margin: 0.000000000+/-0.000000000
Hidden Neurons: 76.90+/-20.42
********* Results ELM-IMA 0.001**************
Acc train: 0.7440+/-0.0359
Acc test: 0.7158+/-0.1046
Iterations: 229.5000+/-27.4235
Iterations IMA: 20.0000